<a href="https://colab.research.google.com/github/nluninja/langchain_agents/blob/main/LangChain_GoogleSearch_Llama33.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

# Install required libraries
!pip install langchain llama python-requests

from langchain.llms import Llama
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.tools import Tool
from langchain.agents import initialize_agent, Tool
import requests

# Define the function for the search tool
def search_web(query):
    """
    Search the web using the Google Custom Search JSON API.
    Replace 'API_KEY' and 'CX_ID' with your actual Google API key and Custom Search Engine ID.
    """
    API_KEY = "your_google_api_key_here"
    CX_ID = "your_google_cse_id_here"
    SEARCH_ENGINE_URL = "https://www.googleapis.com/customsearch/v1"

    response = requests.get(SEARCH_ENGINE_URL, params={"q": query, "key": API_KEY, "cx": CX_ID})
    response.raise_for_status()

    results = response.json()
    # Combine the top results into a single text response
    return "\n".join([item["snippet"] for item in results.get("items", [])])

# Define the tool for the agent
search_tool = Tool(
    name="Web Search",
    func=search_web,
    description="Useful for finding information on the web based on a query."
)

# Initialize LLaMA 3.3 LLM
llama_llm = Llama(
    model_name="llama-3.3",
    api_key="your_llama_api_key_here"
)

# Define a prompt template for answering questions
prompt = PromptTemplate(
    input_variables=["question", "context"],
    template="""
    You are an expert assistant. Use the provided context to answer the user's question.

    Question: {question}
    Context: {context}

    Answer:
    """
)

# Combine the prompt and the LLaMA model into a chain
qa_chain = LLMChain(llm=llama_llm, prompt=prompt)

# Initialize the agent with tools
agent = initialize_agent(
    tools=[search_tool],
    llm=qa_chain,
    agent="zero-shot-react-description",
    verbose=True
)

# Application logic to handle user questions
def answer_question(question):
    """
    Given a user's question, search the web and generate an answer.
    """
    try:
        answer = agent.run(question)
        return answer
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
def main():
    """
    Main function to interact with the user.
    """
    user_question = input("Enter your question: ")
    response = answer_question(user_question)
    print("Answer:", response)

# Run the application
if __name__ == "__main__":
    main()
